In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd     
import plotly           
import plotly.io as pio
import plotly.graph_objects as go
import warnings
import random


warnings.filterwarnings("ignore")

In [4]:
app = racing_bar.Dash()

file_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
df = pd.read_csv(file_url)

df[df['Country/Region']=="China"].head()
df_country = df.groupby('Country/Region').sum().reset_index()
df_country = pd.DataFrame(df_country)

df_country = df_country.drop(["Lat", "Long"], axis=1)

df_wide = df_country.sort_values(by=df_country.columns[-1], ascending=False)
df_wide['Rank'] = round(df_wide[df_wide.columns[-1]].rank(ascending=False))
df_rank = df_wide.drop(df_wide.columns[1:-1], axis=1)
color_list= ["#{:06x}".format(random.randint(0, 0xFFFFFF)) for i in range(181)]
df_rank['Color'] = color_list
df_wide = df_wide.drop(df_wide.columns[-1],axis=1)

date_cols = list(df_wide.columns[1:])
df_long = pd.melt(df_wide, id_vars=['Country/Region'], value_vars=date_cols, var_name='Date', value_name='Confirmed')
df_long['Date'] = pd.to_datetime(df_long['Date'])

df_long = df_long.sort_values(['Country/Region','Date'])

df_long['delta'] = df_long['Confirmed'].diff()
df_long = df_long[df_long['Date']!= "2020-01-22"]
df_long = df_long.merge(df_rank, on = "Country/Region")
df_long.to_csv('/content/drive/My Drive/OPIM 5512 DMBI(Python)/tableau.csv', index=False)





df_long['Date'] = df_long['Date'].dt.strftime('%Y-%m-%d')
df_us = df_long[df_long['Country/Region'] == "US" ]
#df_us['Date'] = df_us['Date'].dt.strftime('%Y-%m-%d')
df_date_list = df_us['Date'].to_list()
df_date_list

keys = pd.DataFrame(range(1,len(df_us['Date'])+1))
keys = keys.rename(columns={0: 'keys'})
keys["keys"] = keys["keys"].astype("object")
key_list = keys['keys'].to_list()


n_frame={}

for d, k in zip(df_date_list, key_list):
    df = df_long[df_long['Date']==d]
    df = df.nlargest(n=10,columns=['Confirmed'])
    df = df.sort_values(by=['Date','Confirmed'])

    n_frame[k]=df

NameError: ignored

In [0]:
fig = go.Figure(
    data=[
        go.Bar(
        x=n_frame[1]['Confirmed'], y=n_frame[1]['Country/Region'],orientation='h',
        text=n_frame[1]['Confirmed'],
        textfont={'size':18}, textposition='inside', insidetextanchor='middle',
        width=0.8,marker={'color':n_frame[1]['Color']})
    ],
    layout=go.Layout(
        xaxis=dict(range=[0, 700], autorange=False, title=dict(text='Total Confirmed Cases',font=dict(size=18))),
        yaxis=dict(range=[-1, 10], autorange=False,tickfont=dict(size=14)),
        title=dict(text='Date:2020-01-23',font=dict(size=28),x=0.5,xanchor='center'),
        # Add button
        updatemenus=[dict(
            type="buttons",
            buttons=[dict(label="Play",
                          method="animate",
                          # https://github.com/plotly/plotly.js/blob/master/src/plots/animation_attributes.js
                          args=[None,
                          {"frame": {"duration": 500, "redraw": True},
                          "transition": {"duration":250,
                          "easing": "linear"}}]
            )]
        )]
    ),
    frames=[
            go.Frame(
                data=[go.Bar(x=value['Confirmed'], y=value['Country/Region'],
                        orientation='h',text = value['Confirmed'], marker={'color':value['Color']}
                        )
                    ],
                layout=go.Layout(
                        xaxis=dict(range=[0, max(value['Confirmed']*1.1)], autorange=False),
                        yaxis=dict(range=[-1, 10], autorange=False,tickfont=dict(size=10)),
                        title=dict(text='Date: '+str(value['Date'].values[0]),
                        font=dict(size=28))
                    )
            )
        for key, value in n_frame.items()
    ]
)

fig.show()

if __name__ == '__main__':
    app.run_server(debug=True)

In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import math
import pandas as pd
import plotly.graph_objs as go

app = dash.Dash()

df = pd.read_csv(
    "https://storage.googleapis.com/learn_pd_like_tidyverse/gapminder.csv")
bubble_size = [math.sqrt(p / math.pi) for p in df["pop"].values]
df['size'] = bubble_size
sizeref = 2*max(df['size'])/(100**2)

app.layout = html.Div([
    html.H2(children='A Gapminder Replica with Dash'),
    dcc.Graph(
        id='gapminder',
        figure={
            'data': [
                go.Scatter(
                    x=df[df['continent'] == i]['gdpPercap'],
                    y=df[df['continent'] == i]['lifeExp'],
                    text=df[df['continent'] == i]['country'],
                    mode='markers',
                    opacity=0.7,
                    marker={
                        'size': df[df['continent'] == i]['size'],
                        'line': {'width': 0.5, 'color': 'white'},
                        'sizeref': sizeref,
                        'symbol': 'circle',
                        'sizemode': 'area'
                    },
                    name=i
                ) for i in df.continent.unique()
            ],
            'layout': go.Layout(
                xaxis={'type': 'log', 'title': 'GDP Per Capita'},
                yaxis={'title': 'Life Expectancy'},
                margin={'l': 40, 'b': 40, 't': 10, 'r': 10},
                legend={'x': 0, 'y': 1},
                hovermode='closest'
            )
        }
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

ModuleNotFoundError: ignored